<a href="https://colab.research.google.com/github/AreebAhmad-02/Rag-using-Langchain-with-Cohere-reranker-and-Zypher7B/blob/main/RAG_implementation_sales_tax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG implementation for Tech-news dataset

### Installation

In [1]:
!pip install -q langchain sentence-transformers cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.1 MB/s eta 0:00:00


In [3]:
!pip install rank_bm25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [4]:
!pip install -U langchain-community -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [7]:
import os
from getpass import getpass

HF_token = getpass()

··········


In [8]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

## Dataset
<!-- [BBC Tech News Dataset](http://mlg.ucd.ie/datasets/bbc.html) -->

In [9]:
directory_path='/content'

batch0preprocessed_file.txt  batch1preprocessed_file.txt  batch2preprocessed_file.txt  sample_data


In [10]:
filenames  = ['/content/batch0preprocessed_file.txt','/content/batch1preprocessed_file.txt','/content/batch2preprocessed_file.txt']
documents = ""
for file in filenames:
  with open(file, 'r') as file:
    # Read the content of the file and append it to the variable
    documents += file.read()


In [11]:
documents[:3]

'CIT'

In [12]:
len(documents)

298846

### Chunking the text

In [13]:
from langchain.schema import Document

In [14]:
filenames  = ['/content/batch0preprocessed_file.txt','/content/batch1preprocessed_file.txt','/content/batch2preprocessed_file.txt']
documents = [] # Initialize documents as a list
for file in filenames:
  with open(file, 'r') as f:
    # Read the content of the file and create a Document object
    documents.append(Document(page_content=f.read()))

text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
text_splits=text_splitter.split_documents(documents)
print(len(text_splits))

642


In [15]:
text_splits[0]

Document(page_content='CITY AND\nUNTY OF SAN FRANCISCO\nCity and County of San Francisco\nCertified Copy\nOrdinance\nCity Hall\n1 Dr. Carlton B. Goodlett Place\nSan Francisco, CA 94102-4689\n[Initiative Ordinance - Business and Tax Regulations Code - Sales Tax for\nTransportation Authority ]\nSponsors: Mandelman; Walton, Peskin, Ronen, Melgar, Stefani, Preston, Mar, Safai\nand Dorsey\nOrdinance approving a new 2022 Transportation Expenditure Plan for the County')

### Embedding Model

In [16]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,
    model_name='BAAI/bge-base-en-v1.5'
)

In [ ]:
# Embed the text splits using the 'embeddings' object
embeddings_result = embeddings.embed_documents([t.page_content for t in text_splits])


In [23]:
embeddings_result

{'error': 'Internal Server Error'}

In [ ]:
vectorstore = FAISS.from_documents(text_splits, embeddings_result)

In [33]:
# Persist the vectorstore
vectorstore.save_local("faiss_index")


In [ ]:
# Load the persisted vectorstore
new_vectorstore = FAISS.load_local("faiss_index", embeddings)

## **Implementing Hybrid Search with ensemble Retrieval**

> Indented block



In [34]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 15})

In [35]:
keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5

In [36]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [37]:
query="Who is exempt from paying the registration fee required by this Article 12, and under what conditions?"

In [42]:
keyword_relevant_doc = keyword_retriever.get_relevant_documents(query)
keyword_relevant_doc

[Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the'),
 Document(page_content="Article 6, the Tax Collector shall determine the amount of the registration fee required based on the\napplicant's estimated gross receipts under Article 12 A1 (Gross Receipts Tax Ordinance) for the\nperiod covered by the registration certificate.\n(ed) Fee for Persons Subject to Administrative Office Tax.\n(1) Except as otherwise provided in this Section 855 and Section 856 of this\nArticle 12, and Section 76.3 of Article 2 of this Business 

In [40]:
relevenat_docs = retriever_vectordb.get_relevant_documents(query)
relevenat_docs

[Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the'),
 Document(page_content="for a registration certificate, using the form prescribed by the Tax Collector. The application\nshall be accompanied by the person's registration fee as determined under this Article 12,\nexcept for the initial application filed for a person that is part of a combined group (as\ndescribed in Section 956.3 of Article 12-A-1) where the combined group has already paid the\nfee on a combined basis. A combined group as described in Section 956.3 

In [64]:
docs_rel=ensemble_retriever.get_relevant_documents(query)
docs_rel

[Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the'),
 Document(page_content="for a registration certificate, using the form prescribed by the Tax Collector. The application\nshall be accompanied by the person's registration fee as determined under this Article 12,\nexcept for the initial application filed for a person that is part of a combined group (as\ndescribed in Section 956.3 of Article 12-A-1) where the combined group has already paid the\nfee on a combined basis. A combined group as described in Section 956.3 

In [67]:
type(docs_rel[0])

langchain_core.documents.base.Document

## Reranking

### Implementing Re-ranking with Cohere-Rerank

In [95]:
Cohere_API_token = getpass()

··········


In [96]:
os.environ["COHERE_API_KEY"] =Cohere_API_token

In [62]:
import cohere
co = cohere.Client(api_key=Cohere_API_token)

In [43]:
from langchain.llms import HuggingFaceHub
model=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-alpha',
                     model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [52]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

In [53]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x790db79b03d0>, top_n=3, model='rerank-english-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x790dbaa2eb30>, search_kwargs={'k': 15}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x790dbaa2e1a0>, k=5)], weights=[0.5, 0.5]))

In [54]:
compressed_docs

[Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the', metadata={'relevance_score': 0.9988883}),
 Document(page_content='September 30, the registration fee shall be 75% of the annual fee; and for persons commencing\nbusiness between October 1 and December 31, the registration fee shall be 50% of the annual fee.\n(52) Notwithstanding any other provision of this Article 12, no person obtaining a\nregistration certificate for a newly established business that qualifies for the minimum\nregistration fee set forth in Section 

In [100]:
!pip install langchain-cohere -q -U

In [108]:
# from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
# from langchain_community.chat_models import ChatCohere
# # from langchain_cohere import ChatCohere

In [ ]:
# cohere_chat_model = ChatCohere(cohere_api_key=Cohere_API_token)
# # cohere_embeddings = CohereEmbeddings(cohere_api_key="{API_KEY}")
# # Create the cohere rag retriever using the chat model
# # llm = ChatCohere(cohere_api_key=Cohere_API_token)
# rag = CohereRagRetriever(llm=cohere_chat_model)

# docs = rag.get_relevant_documents(
#     query,
#     source_documents=compressed_docs,
# )
# # Print the documents
# for doc in docs[:-1]:
#     print(doc.metadata)
#     print("\n\n" + doc.page_content)
#     print("\n\n" + "-" * 30 + "\n\n")
# # Print the final generation
# answer = docs[-1].page_content
# print(answer)
# # Print the final citations
# citations = docs[-1].metadata['citations']
# print(citations)

### rerranking with zypher

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank

compressor = RankLLMRerank(top_n=3, model="zephyr")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compressed_docs = compression_retriever.invoke(query)
pretty_print_docs(compressed_docs)

### MMR (maximal_marginal_relevance) Implementation

In [ ]:
%pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

def maximal_marginal_relevance(sentence_vector, phrases, embedding_matrix, lambda_constant=0.5, threshold_terms=10):
    """
    Return ranked phrases using MMR. Cosine similarity is used as similarity measure.
    :param sentence_vector: Query vector
    :param phrases: list of candidate phrases
    :param embedding_matrix: matrix having index as phrases and values as vector
    :param lambda_constant: 0.5 to balance diversity and accuracy. if lambda_constant is high, then higher accuracy. If lambda_constant is low then high diversity.
    :param threshold_terms: number of terms to include in result set
    :return: Ranked phrases with score
    """
    # todo: Use cosine similarity matrix for lookup among phrases instead of making call everytime.
    s = []
    r = sorted(phrases, key=lambda x: x[1], reverse=True)
    r = [i[0] for i in r]
    while len(r) > 0:
        score = 0
        phrase_to_add = ''
        for i in r:
            first_part = cosine_similarity([sentence_vector], [embedding_matrix.loc[i]])[0][0]
            second_part = 0
            for j in s:
                cos_sim = cosine_similarity([embedding_matrix.loc[i]], [embedding_matrix.loc[j[0]]])[0][0]
                if cos_sim > second_part:
                    second_part = cos_sim
            equation_score = lambda_constant*(first_part)-(1-lambda_constant) * second_part
            if equation_score > score:
                score = equation_score
                phrase_to_add = i
        if phrase_to_add == '':
            phrase_to_add = i
        r.remove(phrase_to_add)
        s.append((phrase_to_add, score))
    return (s, s[:threshold_terms])[threshold_terms > len(s)]

In [ ]:
query_embedding = cohere_client.embed([query]).embeddings[0]


# Extract document contents and metadata
documents = [(doc.page_content, doc.metadata['relevance_score']) for doc in compressed_docs]

# Generate embeddings for the documents
doc_embeddings = {}
for idx, (content, _) in enumerate(documents):
    embedding = cohere_client.embed([content]).embeddings[0]
    doc_embeddings[idx] = embedding

# Create an embedding matrix DataFrame
embedding_matrix = pd.DataFrame.from_dict(doc_embeddings, orient='index')

# Prepare phrases list for MMR
phrases = [(str(idx), score) for idx, (_, score) in enumerate(documents)]

# Apply MMR
ranked_phrases = maximal_marginal_relevance(query_embedding, phrases, embedding_matrix)

# Retrieve and print re-ranked documents
reranked_docs = [list(doc_embeddings.keys())[int(phrase[0])] for phrase in ranked_phrases]

for doc in reranked_docs:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")
Ex

## LLM Prompt

In [68]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well. Based on the context below, answer the user's query.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [69]:
prompt = ChatPromptTemplate.from_template(template)

In [70]:
output_parser = StrOutputParser()

In [71]:
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [59]:
query="Who is exempt from paying the registration fee required by this Article 12, and under what conditions?"

In [72]:
response = chain.invoke(query)

In [81]:
print(response)

Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well. Based on the context below, answer the user's query.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the', metadata={'relevance_score': 0.9988883}), Document(page_content='September 30, the registration fee shall be 75% of the annual fee; and for persons commencing\nbusiness between October 1 and December 31, the registratio

In [91]:
# prompt: how to promt this llm model

# Create a prompt template
prompt_template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well. Based on the context below, answer the user's query.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

# Create a prompt object from the template
prompt = ChatPromptTemplate.from_template(prompt_template)

# Create an output parser to extract the answer from the model's response
output_parser = StrOutputParser()

# Create a chain that combines the prompt, the model, and the output parser
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

# Use the chain to invoke the model with a query
query = "Who is exempt from paying the registration fee required by this Article 12, and under what conditions?"
response = chain.invoke(query)

# Print the model's response
print(response)


Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well. Based on the context below, answer the user's query.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content='Revenue Code of 1986, as amended, as qualified by Sections 502, 503, 504, and 508 of the\nInternal Revenue Code of 1986, as amended, shall be exempt from the registration fee under\nthis Article 12, only so long as those exemptions continue to exist under state or federal law.\n29SANEY CO\n2024 MA 10 PM 1:29\n(+) A person shall be exempt from paying the registration fee required by this\nARTIENT OF ELEC\nSection 855 if and to the extent that, and only so long as, federal or state law prohibits the', metadata={'relevance_score': 0.9988883}), Document(page_content='September 30, the registration fee shall be 75% of the annual fee; and for persons commencing\nbusiness between October 1 and December 31, the registratio